In [2]:
%matplotlib inline
import torch

from botorch.fit import fit_gpytorch_model
from botorch.models import SingleTaskGP
from gpytorch.mlls import ExactMarginalLogLikelihood

### Fit a GP model to data

In [14]:
train_X = torch.rand(10,2)
train_X

tensor([[0.0597, 0.0678],
        [0.9568, 0.9647],
        [0.0029, 0.7403],
        [0.7849, 0.7753],
        [0.0590, 0.2358],
        [0.5073, 0.7788],
        [0.3033, 0.7137],
        [0.5321, 0.6660],
        [0.1362, 0.1604],
        [0.1395, 0.3266]])

In [16]:
Y = 1 - (train_X - 0.5).norm(dim=-1, keepdim=True) 
Y

tensor([[0.3831],
        [0.3484],
        [0.4479],
        [0.6038],
        [0.4859],
        [0.7211],
        [0.7095],
        [0.8309],
        [0.5024],
        [0.6000]])

In [18]:
Y += 0.1 * torch.rand_like(Y)
Y

tensor([[0.4439],
        [0.4319],
        [0.4494],
        [0.6842],
        [0.5216],
        [0.7456],
        [0.7612],
        [0.8403],
        [0.5733],
        [0.6085]])

In [20]:
train_Y = (Y - Y.mean()) / Y.std()
train_Y

tensor([[-1.1044],
        [-1.1859],
        [-1.0670],
        [ 0.5330],
        [-0.5749],
        [ 0.9512],
        [ 1.0572],
        [ 1.5964],
        [-0.2227],
        [ 0.0171]])

In [25]:
gp = SingleTaskGP(train_X, train_Y)
mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
fit_gpytorch_model(mll);


### Construct an acquisition func

In [26]:
from botorch.acquisition import UpperConfidenceBound

UCB = UpperConfidenceBound(gp, beta=0.1)

### Optimize an acquisition function

In [27]:
from botorch.optim import optimize_acqf

bounds = torch.stack([torch.zeros(2), torch.ones(2)])
candidate, acq_value = optimize_acqf(
    UCB, bounds=bounds, q=1, num_restarts=5, raw_samples=20,
)

In [28]:
candidate

tensor([[0.5468, 0.4648]])

In [30]:
acq_value

tensor(2.0427)

In [31]:
bounds

tensor([[0., 0.],
        [1., 1.]])